In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import os
import requests
from bs4 import BeautifulSoup, Comment
from datetime import datetime
import html5lib
import lxml

from utils import mp_to_minutes

In [2]:
# extract draft class from the url
def load_draft_class(draft_year):
    url = f"https://www.basketball-reference.com/draft/NBA_{draft_year}.html"

    df = pd.read_html(url)[0]
    df_links = pd.read_html(url, extract_links="body")[0]

    df.columns = [f"{a}_{b}".strip("_") for a, b in df.columns]
    df_links.columns = df.columns

    df = df[df["Round 1_Player"].notna()].copy()

    df["player"] = df["Round 1_Player"]
    df["player_id"] = df_links["Round 1_Player"].apply(
        lambda x: x[1].split("/")[-1].replace(".html", "")
        if isinstance(x, tuple) and x[1]
        else None
    )

    # 🔑 drop header rows like "Round 2"
    df = df[df["player_id"].notna()].copy()

    df["draft_year"] = draft_year
    df["rookie_season"] = draft_year + 1

    return df[["draft_year", "player", "player_id", "rookie_season"]]



In [3]:
# load or scrape draft class with caching
# def load_or_scrape_draft(year):
#     path = f"assets/drafts/draft_{year}.csv"
#     if os.path.exists(path):
#         return pd.read_csv(path)
    
#     df = load_draft_class(year)
#     df.to_csv(path, index=False)
#     return df

# function to scrape a single draft class
def scrape_and_save_year(year):
    df = load_draft_class(year)
    df.to_csv(f"assets/drafts/draft_{year}.csv", index=False)
    print(f"✓ saved {year}")


In [3]:
# freeze the draft years collected
import glob
draft_classes = pd.concat(
    [pd.read_csv(f) for f in glob.glob("assets/drafts/draft_20*.csv")],
    ignore_index=True
)
draft_classes.head(100)
draft_classes.to_csv("assets/draft_classes.csv", index=False)


In [4]:
# override rookie seasons for specific players
rookie_overrides = {
    "griffbl01": 2011,  # drafted 2009, rookie season 2010–11
    "embiijo01": 2016,  # drafted 2014, rookie season 2015-16
}

In [5]:
def load_rookie_gamelog(player_id, season):
    first_letter = player_id[0]
    url = (
        f"https://www.basketball-reference.com/players/"
        f"{first_letter}/{player_id}/gamelog/{season}"
    )

    df = pd.read_html(
        url,
        attrs={"id": "player_game_log_reg"}
    )[0]

    # 1️⃣ KEEP ONLY REAL GAMES (must have an opponent)
    df = df[df["Opp"].notna()].copy()

    # 2️⃣ Convert MP to numeric (DNPs become NaN)
    df["MP"] = df["MP"].apply(mp_to_minutes)

    # 3️⃣ Drop games not played
    df = df[df["MP"].notna()].copy()

    return df


In [6]:
def aggregate_capped_minutes(
    gamelog,
    cap_minutes=900,
    stat_cols=("PTS", "TRB", "AST", "STL", "BLK", "TOV"),
):
    """
    Aggregate rookie production up to a capped number of minutes.
    """

    # Ensure counting stats are numeric
    gamelog = gamelog.copy()
    for stat in stat_cols:
        gamelog[stat] = pd.to_numeric(gamelog[stat], errors="coerce")

    total_minutes = 0.0
    totals = {stat: 0.0 for stat in stat_cols}

    for _, row in gamelog.iterrows():
        mp = row["MP"]

        if total_minutes >= cap_minutes:
            break

        remaining = cap_minutes - total_minutes

        if mp <= remaining:
            weight = 1.0
            used_mp = mp
        else:
            weight = remaining / mp
            used_mp = remaining

        total_minutes += used_mp

        for stat in stat_cols:
            if pd.notna(row[stat]):
                totals[stat] += row[stat] * weight

    return {
        "minutes_used": total_minutes,
        **totals,
    }

In [7]:
# calculate rate stats
def add_rate_stats(agg, minutes_base=36):
    factor = minutes_base / agg["minutes_used"]
    return {
        f"{k}_per_{minutes_base}": v * factor
        for k, v in agg.items()
        if k not in ("minutes_used",)
    }

In [9]:
rookie_caps_900 = pd.read_csv("assets/rookie_caps_900.csv")
errors_df = pd.read_csv("assets/rookie_errors.csv")

In [9]:
len(rookie_caps_900) - len(errors_df)

-2816

In [10]:
# retry = errors_df[errors_df["error"].str.contains("429")]
# retry.head()

In [11]:
# new_results = []
# new_errors = []
# cap = 900

# for _, row in retry.iterrows():
#     try:
#         gamelog = load_rookie_gamelog(row.player_id, row.rookie_season)
#         agg = aggregate_capped_minutes(gamelog, cap_minutes=cap)

#         new_results.append({
#             "player_id": row.player_id,
#             "rookie_season": row.rookie_season,
#             **agg
#         })

#         time.sleep(6)  # polite delay

#     except Exception as e:
#         new_errors.append({
#             "player_id": row.player_id,
#             "rookie_season": row.rookie_season,
#             "error": str(e),
#         })


In [12]:
# rookie_caps_900 = pd.concat([rookie_caps_900, pd.DataFrame(new_results)], ignore_index=True)
# errors_df = pd.concat([errors_df, pd.DataFrame(new_errors)], ignore_index=True)

In [13]:
# reconstruct missing player names and draft years
# rookie_caps_900 = rookie_caps_900.merge(
#     draft_classes,
#     on=["player_id", "rookie_season"],
#     how="left",
#     suffixes=("", "_fix"),
# )

# rookie_caps_900["draft_year"] = rookie_caps_900["draft_year"].fillna(rookie_caps_900["draft_year_fix"])
# rookie_caps_900["player"] = rookie_caps_900["player"].fillna(rookie_caps_900["player_fix"])

# rookie_caps_900 = rookie_caps_900.drop(columns=[c for c in rookie_caps_900.columns if c.endswith("_fix")])
# rookie_caps_900.head(200)


In [14]:
# # deduplicate to keep only one entry per player per rookie season
# rookie_caps_900 = (
#     rookie_caps_900
#     .sort_values("minutes_used", ascending=False)
#     .drop_duplicates(subset=["player_id", "rookie_season"], keep="first")
#     .reset_index(drop=True)
# )

In [15]:
# rookie_caps_900.shape
# rookie_caps_900["rookie_season"].min(), rookie_caps_900["rookie_season"].max()


In [16]:
# rookie_caps_900.duplicated(subset=["player_id", "rookie_season"]).sum()

In [18]:
# build todo dataframe
mask_done = draft_classes.apply(
    lambda r: (r.player_id, r.rookie_season) in already_done,
    axis=1
)

mask_failed = draft_classes.apply(
    lambda r: (r.player_id, r.rookie_season) in already_failed,
    axis=1
)

todo = draft_classes[~mask_done & ~mask_failed].copy()

In [19]:
print("Total draft classes:", len(draft_classes))
print("Already done:", len(already_done))
print("Already failed:", len(already_failed))
print("Remaining todo:", len(todo))

todo.head()

retry = draft_classes.merge(
    errors_df[["player_id", "rookie_season"]],
    on=["player_id", "rookie_season"],
    how="inner"
)

len(retry), retry.head()

Total draft classes: 1542
Already done: 127
Already failed: 1529
Remaining todo: 0


(2943,
    draft_year              player  player_id  rookie_season
 0        2000         Etan Thomas  thomaet01           2001
 1        2000  Courtney Alexander  alexaco02           2001
 2        2000      Mateen Cleaves  cleavma01           2001
 3        2000       Jason Collier  collija02           2001
 4        2000       Hedo Türkoğlu  turkohe01           2001)

In [20]:
new_results = []
new_errors = []

for i, (_, row) in enumerate(retry.iterrows(), 1):
    try:
        gamelog = load_rookie_gamelog(row.player_id, row.rookie_season)
        agg = aggregate_capped_minutes(gamelog, cap_minutes=900)

        new_results.append({
            **row.to_dict(),
            **agg
        })

        if i % 10 == 0:
            print(f"Processed {i}/{len(retry)}")

        time.sleep(6)

    except Exception as e:
        new_errors.append({
            "player_id": row.player_id,
            "rookie_season": row.rookie_season,
            "error": str(e),
        })

Processed 20/2943
Processed 30/2943


In [21]:
len(new_results)

30

In [22]:
if new_results:
    new_df = pd.DataFrame(new_results)

    rookie_caps_900 = (
        pd.concat([rookie_caps_900, new_df], ignore_index=True)
        .sort_values("minutes_used", ascending=False)
        .drop_duplicates(subset=["player_id", "rookie_season"], keep="first")
        .reset_index(drop=True)
    )

if new_errors:
    errors_df = (
        pd.concat([errors_df, pd.DataFrame(new_errors)], ignore_index=True)
        .drop_duplicates(subset=["player_id", "rookie_season"])
        .reset_index(drop=True)
    )


In [27]:
existing = set(zip(rookie_caps_900.player_id, rookie_caps_900.rookie_season))
new_keys = [(r["player_id"], r["rookie_season"]) for r in new_results]

sum(k in existing for k in new_keys), len(new_keys)

(30, 30)

In [54]:
# new completed and failed key sets
done_keys = set(zip(rookie_caps_900.player_id, rookie_caps_900.rookie_season))
failed_keys = set(zip(errors_df.player_id, errors_df.rookie_season))

In [55]:
list(done_keys)[:5], list(failed_keys)[:5]


([('nachbbo01', 2003),
  ('satteke01', 2002),
  ('salmojo01', 2003),
  ('johnsde03', 2001),
  ('turkohe01', 2001)],
 [('thomaet01', 2001)])

In [56]:
("flaggco01", 2026) in done_keys, ("flaggco01", 2026) in failed_keys


(False, False)

In [57]:
# rebuild todo dataframe safely
draft_classes["_key"] = list(zip(draft_classes.player_id, draft_classes.rookie_season))

draft_classes["_done"] = draft_classes["_key"].isin(done_keys)
draft_classes["_failed"] = draft_classes["_key"].isin(failed_keys)

todo = draft_classes[~draft_classes["_done"] & ~draft_classes["_failed"]].copy()

len(todo), todo.head()


(1414,
     draft_year           player  player_id  rookie_season               _key  \
 19        2000   Speedy Claxton  claxtsp01           2001  (claxtsp01, 2001)   
 26        2000    Primož Brezec  brezepr01           2001  (brezepr01, 2001)   
 29        2000      Marko Jarić  jaricma01           2001  (jaricma01, 2001)   
 40        2000  Chris Carrawell  carrach01           2001  (carrach01, 2001)   
 45        2000  DeeAndre Hulett  huletde01           2001  (huletde01, 2001)   
 
     _done  _failed  
 19  False    False  
 26  False    False  
 29  False    False  
 40  False    False  
 45  False    False  )

In [61]:
new_results = []
new_errors = []

for i, (_, row) in enumerate(todo.iterrows(), 1):
    try:
        gamelog = load_rookie_gamelog(row.player_id, row.rookie_season)
        agg = aggregate_capped_minutes(gamelog, cap_minutes=900)

        new_results.append({
            **row.to_dict(),
            **agg
        })

        if i % 25 == 0:
            print(f"Processed {i}/{len(todo)}")

        time.sleep(6)

    except Exception as e:
        new_errors.append({
            "player_id": row.player_id,
            "rookie_season": row.rookie_season,
            "error": str(e),
        })

In [62]:
if new_results:
    rookie_caps_900 = pd.concat([rookie_caps_900, pd.DataFrame(new_results)], ignore_index=True)
    rookie_caps_900 = rookie_caps_900.drop_duplicates(subset=["player_id", "rookie_season"])

if new_errors:
    errors_df = pd.concat([errors_df, pd.DataFrame(new_errors)], ignore_index=True)
    errors_df = errors_df.drop_duplicates(subset=["player_id", "rookie_season"])

In [64]:
len(new_results), len(new_errors)


(0, 1414)

In [52]:
# rebuild errors dataframe
errors_df = errors_df[~errors_df["error"].str.contains("429", na=False)]
errors_df = errors_df.drop_duplicates(subset=["player_id", "rookie_season"])
errors_df.to_csv("assets/rookie_errors_clean.csv", index=False)

In [53]:
errors_df = pd.read_csv("assets/rookie_errors_clean.csv")

In [ ]:
# batch process rookie classes
# cap = 900
# results = []
# errors = []

# for _, row in draft_classes.iterrows():
#     try:
#         gamelog = load_rookie_gamelog(row.player_id, row.rookie_season)
#         agg = aggregate_capped_minutes(gamelog, cap_minutes=cap)

#         results.append({
#             **row.to_dict(),
#             **agg
#         })

#     except Exception as e:
#         errors.append({
#             "player_id": row.player_id,
#             "rookie_season": row.rookie_season,
#             "error": str(e),
#         })

# rookie_caps_900 = pd.DataFrame(results)
# errors_df = pd.DataFrame(errors)